# Generative Adversarial Networks (GANs)

Ara an alternative to VAEs for learning latent spaces of images. They enbale the generation of fairly realistic synthetic images by forcing the generated images to be statistically almost indistinguishable from real ones.

An intuitive way to understand GANs is to imagine a forger trying to create a fake Picasso painting. At first, the forger is pretty bad at the task. He mixes some of his fakes with authentic Picassos and shows them all to an art dealer. The art dealer makes an authenticity assessement for each Picasso and gives feedback about what makes a Picasso look like a real Picasso. The forger goes to his studio to prepare more fakes. As time goes the forger becomes increasingly expert at imitating pictures, and the art dealer becomes increasingly expert spotting fakes

A GAN is made of two parts:

- *Generator network*: Takes as input a random vector (random point in the latent space), and decodes it into a synthetic image.

- *Discriminator network (or adversary)*: Takes as input an image (real or fake), and predicts weather the image came from the training set or was created by the *generator*.

![](https://cdn-images-1.medium.com/max/1200/1*pHOkZ0HJrUSP827-fZFETg.png)

## Schematic implementation of *Deep Convolutional GAN (DCGAN)*

> In this example will be using the CIFAR-10 dataset

1. A `generator` networks maps vectors of shape `(latent_dim, )` to images of shape `(32, 32, 3)`.

2. A `discriminator` network maps images ti a binary score estimating the probability thet the image is real.

3. A `gan` network chains the generator and the discriminator together: `gan(x) = discriminator(generator(x))`. Thus this `gan` maps latent space vector to the discriminator's assessement of the realism of this latent vectors as decoded by the generator.

4. You train the discriminator using examples of real and fake images along with "real" / "fake" labels, just as you train any regular image-classification model.

5. To train the generator, you use the gradients of the generator's weights with regard to the loss of the `gan` model. This means, at every step, you move the weights of the generator in a direction that makes the discriminator more likely to classify as real the images decoded by the generator. In other words, you train the generator to fool the discriminator.

## GANs' tricks

The process of training GANs adn tuning GAN implementation is difficult.

- We use `tanh` as the activation in the generator, instead of `sigmoid`.
- We sample points from the latent space using a *normal distribution*, not a uniform dits.
- Stochasticity is good to introduce robustness. GANs are likely to stuck in all sort of ways, introducing randomness helps prevent this. We introduce randomness using dropout and adding noise to the labels for the discriminator.
- Sparsisity is often a desirable property, but not in GANs. Two things can include gradient sparsisity: ReLU and max pooling. Insead of ReLU we will be using LeakyReLU which reduces sparsisity by allowing small negative numbers, and instead of max pooling will use convolutional strided layers.
- In generted images, it's common to see checkerboards artifacts caused by unequal coverage of the picel space in the generator. To fix this, we use a kernel size that's diviseible by the stride size whenever we uses an strided convolutional layer.

![Checkerboards](https://distill.pub/2016/deconv-checkerboard/thumbnail.jpg)

## The generator

In [1]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = layers.Input((latent_dim, ))

# Generate a 16x16x128 feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upscale to 32x32 using a conv2d transpose
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produces 32 x 32 x 3 feature map (shape of CIFAR 10 image)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv

## The discriminator

In [2]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dropout(.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
___________________________

In [3]:
discriminator_optimizer = keras.optimizers.RMSprop(
                                            lr=0.0008,
                                            clipvalue=1.,
                                            decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

## The adversarial network

In [4]:
discriminator.trainable = False

gan_input = layers.Input(shape=(latent_dim, ))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## How to train DCGAN

For each epoch:

1. Draw random points in the latent space.
2. Generate images with the `generator` using this random noise.
3. Mix the generated images with real ones.
4. Train `discriminator` using this mixed images, with corresponding targets: either "real (0)" or "fake (1)"
5. Draw new random points in the latent space.
6. Train `gan` using this random vectors, with targets that all say "these are real images". this only updets the generator weights because at gan compile time discriminator weights were frozen. This trains the generator to fool the discriminator.

In [ ]:
import os
from keras.preprocessing import image

# Load CIFAR10 data
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

# Select frog images (class 6)
x_train = x_train[y_train.flatten() == 6]

# Normalize data
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = '.'

# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)

    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # Occasionally save / plot
    if step % 100 == 0:
        # Save model weights
        gan.save_weights('gan.h5')

        # Print metrics
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        # Save one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

        # Save one real image, for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

Instructions for updating:
Use tf.cast instead.


/home/guillem/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss at step 0: 0.6887509
adversarial loss at step 0: 0.23421034
discriminator loss at step 100: 8.464162
adversarial loss at step 100: 1.0000001e-07
discriminator loss at step 200: 8.416028
adversarial loss at step 200: 1.0000001e-07


When training, you may see the adversarial loss begin to increase considerabilym while the discriminative loss tends to zero. The discriminator may end up dominating the generator. If that's the case, try reducing the discriminator learning rate and increase the droput rate of the discriminator